# 重回帰分析
人工データを用いた重回帰モデルの求め方を説明する<br>
- statsmodels.formula.api.ols https://www.statsmodels.org/dev/generated/statsmodels.formula.api.ols.html<br>
- API Reference https://www.statsmodels.org/stable/api.html<br>
- Fitting models using R-style formulas https://www.statsmodels.org/stable/example_formulas.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline

## F検定の値を見る
真のシステム：y = b1*x1 + b2*x2 + noise  
モデル:　y = b1*x1 + b2*x2

In [ ]:
np.random.seed(123) #再現性を得るため
N = 30
noise = np.random.normal(0.0, 0.1, N)
X = np.linspace(-np.pi,np.pi,N)

b1, b2 = 1.1, -0.55 # beta_1, beta_2
x1 = 0.4*X
x2 = np.sin(X)
y = b1*x1 + b2*x2 + noise

In [ ]:
fig = plt.subplots(figsize=(5,3))
plt.plot(x1, color='k', label='x1', linestyle='dotted')
plt.plot(x2, color='gray', label='x2', linestyle='dashed')
plt.plot(y, color='k', label='y')
plt.xlabel('X')
plt.ylabel('x1, x2, y')
plt.legend()
plt.grid()
#plt.savefig('REG_Multiple_01.pdf', bbox_inches='tight')
plt.show()

In [ ]:
df = pd.DataFrame({'y':y, 'x1':x1, 'x2':x2})
results = smf.ols('y ~ x1 + x2 -1', data=df).fit()
results.summary()

#### 偏回帰係数を変えたシミュレーション

In [ ]:
b1, b2 = 0.1, -0.05
y = b1*x1 + b2*x2 + noise
df = pd.DataFrame({'y':y, 'x1':x1, 'x2':x2})
results = smf.ols('y ~ x1 + x2 -1', data=df).fit()
results.summary()

In [ ]:
fig = plt.subplots(figsize=(5,3))
plt.plot(x1, color='k', label='x1', linestyle='dotted')
plt.plot(x2, color='gray', label='x2', linestyle='dashed')
plt.plot(y, color='k', label='y')
plt.xlabel('X')
plt.ylabel('x1, x2, y')
plt.legend()
plt.grid()
#plt.savefig('REG_Multiple_02.pdf', bbox_inches='tight')
plt.show()

## 多重共線性（multicollinearity）の影響を見る
x2とx3が非常に強い相関を持つ場合を扱う。<br>
df3 : y = b1*x1 + b2*x2 + b3*x3

In [ ]:
np.random.seed(456) #再現性を得るため
b1, b2, b3 = 1.5, 3.1, 0.5
x3 = x2 + np.random.normal(0.0, 0.01, N)
y = b1*x1 + b2*x2 + b3*x3 + noise
df3 = pd.DataFrame({'y':y, 'x1':x1, 'x2':x2, 'x3':x3})

#### x2とx3の相関を見る
相関が非常に高いことが視認できる。

In [ ]:
fig = plt.subplots(figsize=(3,2))
plt.scatter(b2*x2, b3*x3)
plt.grid()
#相関係数の計算
np.corrcoef(b2*x2, b3*x3)

#### y, x1, x2, x3のプロット

In [ ]:
fig = plt.subplots(figsize=(5,3))
plt.plot(b1*x1, color='k', label='x1', linestyle='dotted')
plt.plot(b2*x2, color='b', label='x2', linestyle='dashed')
plt.plot(b3*x3, color='r', label='x3', linestyle='dashdot')
plt.plot(y, color='k', label='y', linestyle='solid')
plt.xlabel('data number')
plt.ylabel('x1, x2, x3, y')
plt.legend()
plt.grid()
#plt.savefig('REG_Multiple_03.pdf', bbox_inches='tight')
plt.show()

In [ ]:
results = smf.ols('y ~ x1 + x2 + x3 -1', data=df3).fit()
results.summary()

## 注意：
上記のNotesで，次のメッセージが出た場合<br>
The condition number is large, 1.72e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"The condition number is large"というWarningメッセージが出た場合<br>
condition number（条件数）は、行列の固有値から計算される数値であり、この値が大きいほど連立方程式が解きにくくなり、<br>
解に誤差を含む可能性が高いことを示唆する。したがって、出力された数値解の信頼度はかなり低いと考えて良い。

#### モデルの変更 y  = b1*x1 + b2*x2

In [ ]:
results = smf.ols('y ~ x1 + x2 -1', data=df3).fit()
results.summary()

#### x1, x2を用いて予測を行う

In [ ]:
b1, b2 = results.params
n1, n2 = 5, 15
xx1 = np.array([x1[n1], x1[n2]])
xx2 = np.array([x2[n1], x2[n2]])
print(xx1)
print(xx2)
ypred =b1*xx1 + b2*xx2
ypred

In [ ]:
fig = plt.subplots(figsize=(5,3))
plt.plot(b1*x1, color='k', label='x1', linestyle='dotted')
plt.plot(b2*x2, color='b', label='x2', linestyle='dashed')
plt.plot(x3, color='r', label='x3', linestyle='dashdot')
plt.plot(y, color='k', label='y', linestyle='solid')

plt.scatter(n1, ypred[0])
plt.scatter(n2, ypred[1])

plt.xlabel('data number')
plt.ylabel('x1, x2, x3, y')
plt.legend()
plt.grid()
#plt.savefig('REG_Multiple_04.pdf', bbox_inches='tight')
plt.show()

## ３Dプロット
グラフをマウスを用いて回転させる例：
- 次のセル内のmatplotlib notebookのコメントを外すこと<br>
- 参照　https://matplotlib.org/mpl_toolkits/mplot3d/tutorial.html

In [ ]:
from mpl_toolkits.mplot3d import Axes3D # for 3D-graph
#%matplotlib notebook

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 軸ラベルの設定
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('y')

# 表示範囲の設定
ax.set_xlim(-5, 5)
ax.set_ylim(-4, 4)
ax.set_zlim(-6, 10)

#観測データのプロット
ax.scatter(x1, x2, y, s=10, color='green')

plt.show()